In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [29]:
df = pd.read_csv('book_match_model.csv', sep=';') 

enfant :  inferieur a 12 
ado : 12-17
jeune aldulte  : 18-25
 senior : 51 et plus 

In [30]:
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Genre Principal,Genres Secondaires,Image-URL-L,Moyenne_Adolescent 12-17,Moyenne_Adulte 26-35,Moyenne_Adulte 36-50,Moyenne_Enfant 1-12,Moyenne_Jeune adulte 18-25,Moyenne_Senior 50+
0,000617695X,The Great & Secret Show,Barker Clive,1990-01-01,HarperCollins Publishers,Horror,"Fantasy, Fiction, Fantasy, Dark Fantasy, Thril...",http://images.amazon.com/images/P/000617695X.0...,NaN,6.00,NaN,NaN,NaN,NaN
1,0007144350,Collins Complete Works of Oscar Wilde (Collins...,Oscar Wilde,2003-01-01,HarperCollins Publishers,Classics,"Fiction, Poetry, Plays, Short Stories, Literat...",http://images.amazon.com/images/P/0007144350.0...,NaN,NaN,10.000000,NaN,NaN,NaN
2,0020199856,LOVE OF THE LAST TYCOON,F. Scott Fitzgerald,1995-01-01,Scribner,Classics,"Fiction, Literature, Literature, American, Nov...",http://images.amazon.com/images/P/0020199856.0...,NaN,8.00,NaN,NaN,NaN,NaN
3,0020360754,Heart Songs and Other Stories,Annie Proulx,1995-01-01,Scribner,Short Stories,"Fiction, Literary Fiction, Literature, America...",http://images.amazon.com/images/P/0020360754.0...,NaN,8.50,5.000000,NaN,NaN,NaN
4,0020421907,Seaward,Susan Cooper,1987-01-01,Simon Pulse,Fantasy,"Young Adult, Fiction, Childrens, Adventure, Ch...",http://images.amazon.com/images/P/0020421907.0...,NaN,8.00,NaN,NaN,9.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442,8479019395,Amarse Con Los Ojos Abiertos / To Love With Ey...,Jorge Bucay,2003-01-01,Alfaguara/Santillana USA,Psychology,"Fiction, Self Help, Relationships, Philosophy,...",http://images.amazon.com/images/P/8479019395.0...,NaN,5.50,NaN,NaN,NaN,NaN
2443,8483101610,"Soldados De Salamina (Coleccion Andanzas, 433)",Javier Cercas,2001-01-01,Tusquets,Fiction,"Cultural, Spain, European Literature, Spanish ...",http://images.amazon.com/images/P/8483101610.0...,NaN,9.00,8.666667,NaN,6.0,NaN
2444,8484504794,El Misterio De Salem's Lot/Salem's Lot,Stephen King,2003-01-01,"Plaza & Janes Editores, S.A.",Horror,"Fiction, Paranormal, Vampires, Fantasy, Thrill...",http://images.amazon.com/images/P/8484504794.0...,NaN,NaN,NaN,NaN,9.0,NaN
2445,8495618605,El Codigo Da Vinci / The Da Vinci Code,Dan Brown,2003-01-01,Ediciones Urano,Fiction,"Mystery, Thriller, Thriller, Mystery Thriller,...",http://images.amazon.com/images/P/8495618605.0...,6.0,5.75,7.666667,NaN,9.0,NaN


In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor


MODELE KNN

In [32]:
# Copie du dataframe de base
df = df.copy()

# Nettoyage des genres
df['Genres Secondaires'] = df['Genres Secondaires'].fillna('')
df['Genre Principal'] = df['Genre Principal'].fillna('')

# Combinaison des genres principaux et secondaires
df['genre_combined'] = (df['Genre Principal'] + '|' + df['Genres Secondaires']).str.lower().str.split('[,|]')


In [33]:
# Encodage des genres avec MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre_combined'])


In [34]:
def get_age_group(age):
    if age <= 12:
        return 'Moyenne_Enfant 1-12'
    elif age <= 17:
        return 'Moyenne_Adolescent 12-17'
    elif age <= 25:
        return 'Moyenne_Jeune adulte 18-25'
    elif age <= 35:
        return 'Moyenne_Adulte 26-35'
    elif age <= 50:
        return 'Moyenne_Adulte 36-50'
    else:
        return 'Moyenne_Senior 50+'


In [35]:
def get_model_for_age_group(age_group_column):
    # On filtre les lignes qui ont une note dans cette tranche d’âge
    mask = ~df[age_group_column].isna()
    X = genre_encoded[mask]
    y = df.loc[mask, age_group_column].values

    model = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=5))
    model.fit(X, y)
    return model


In [36]:
def recommander_livres_knn_supervise(user_age, user_genres_principaux, user_genres_secondaires=[], top_k=5):
    # Déterminer la colonne cible correspondant à l'âge de l'utilisateur
    age_group_col = get_age_group(user_age)

    # Vérifier qu'il y a suffisamment de données
    if df[age_group_col].dropna().shape[0] < 5:
        raise ValueError(f"Pas assez de données pour la tranche d'âge {age_group_col}.")

    # Entraîner le modèle pour cette tranche
    model = get_model_for_age_group(age_group_col)

    # Préparer les genres utilisateur
    user_genres = [g.lower() for g in user_genres_principaux + user_genres_secondaires]
    user_vector = mlb.transform([user_genres])
    user_input = user_vector.reshape(1, -1)

    # Prédire la note pour tous les livres de la tranche d’âge
    mask_valid = ~df[age_group_col].isna()
    X_all = genre_encoded[mask_valid]
    df_valid = df[mask_valid].copy()
    df_valid['predicted_rating'] = model.predict(X_all)

    # Filtrer les livres qui ont au moins un genre en commun
    df_valid = df_valid[df_valid['genre_combined'].apply(lambda g: any(gen in g for gen in user_genres))]

    # Trier par prédiction
    top_books = df_valid.sort_values(by='predicted_rating', ascending=False).head(top_k)

    return top_books[['Book-Title', 'Book-Author', 'Publisher', age_group_col, 'predicted_rating']]


In [37]:
user_age = 30
user_genres = ['Fiction']
user_sous_genres = ['Thriller']

recommandations = recommander_livres_knn_supervise(user_age, user_genres, user_sous_genres)
recommandations


,Book-Title,Book-Author,Publisher,Moyenne_Adulte 26-35,predicted_rating
1072,Holes,LOUIS SACHAR,Yearling,9.5,9.640909
176,Betsy and Tacy Go Over the Big Hill (Betsy-Tacy),Maud Hart Lovelace,HarperTrophy,10.0,9.600000
183,Betsy and Joe (Betsy & Tacy),Maud Hart Lovelace,HarperTrophy,10.0,9.600000
193,Emily of Deep Valley (Deep Valley Book),Maud Hart Lovelace,HarperTrophy,10.0,9.600000
2216,My Name Is Asher Lev,Chaim Potok,Anchor Books/Doubleday,10.0,9.400000


 Ridge

In [38]:
from sklearn.linear_model import Ridge


In [39]:
def get_ridge_model_for_age_group(age_group_column, alpha=1.0):
    # On filtre les livres avec une note dans cette tranche d’âge
    mask = ~df[age_group_column].isna()
    X = genre_encoded[mask]
    y = df.loc[mask, age_group_column].values

    model = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
    model.fit(X, y)
    return model


In [40]:
def recommander_livres_ridge(user_age, user_genres_principaux, user_genres_secondaires=[], top_k=5):
    age_group_col = get_age_group(user_age)

    if df[age_group_col].dropna().shape[0] < 5:
        raise ValueError(f"Pas assez de données pour la tranche d'âge {age_group_col}.")

    # Entraîner le modèle Ridge
    model = get_ridge_model_for_age_group(age_group_col)

    # Préparation des genres utilisateur
    user_genres = [g.lower() for g in user_genres_principaux + user_genres_secondaires]
    user_vector = mlb.transform([user_genres])
    user_input = user_vector.reshape(1, -1)

    # Prédiction pour tous les livres valides
    mask_valid = ~df[age_group_col].isna()
    X_all = genre_encoded[mask_valid]
    df_valid = df[mask_valid].copy()
    df_valid['predicted_rating'] = model.predict(X_all)

    # Filtrer selon les genres de l’utilisateur
    df_valid = df_valid[df_valid['genre_combined'].apply(lambda g: any(gen in g for gen in user_genres))]

    # Trier les meilleurs livres
    top_books = df_valid.sort_values(by='predicted_rating', ascending=False).head(top_k)

    return top_books[['Book-Title', 'Book-Author', 'Publisher', age_group_col, 'predicted_rating']]


In [41]:
user_age = 30
user_genres = ['Fiction']
user_sous_genres = ['Thriller']

recommandations_ridge = recommander_livres_ridge(user_age, user_genres, user_sous_genres)
recommandations_ridge


,Book-Title,Book-Author,Publisher,Moyenne_Adulte 26-35,predicted_rating
559,Miracle on the 17th Green : A Novel about Life...,James Patterson,Back Bay Books,10.0,10.508931
1442,Lila: An Inquiry Into Morals,Robert M. Pirsig,Bantam Books,9.0,10.234139
1919,Judy Moody Saves the World (Judy Moody),Megan McDonald,Candlewick Press (MA),10.0,10.102713
2343,The Lighthouse at the End of the World,Jules Verne,Fredonia Books (NL),10.0,9.998467
1107,Chorus Skating,Alan Dean Foster,Aspect,10.0,9.998447


COMPARAISSON 

In [42]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


In [43]:
def split_data_by_age_group(age_group_col, test_size=0.2, random_state=42):
    mask = ~df[age_group_col].isna()
    X = genre_encoded[mask]
    y = df.loc[mask, age_group_col].values

    return train_test_split(X, y, test_size=test_size, random_state=random_state)


In [44]:
def evaluate_model(model, X_test, y_test, model_name=''):
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"📊 Résultats {model_name}")
    print(f"RMSE : {rmse:.3f}")
    print(f"MAE  : {mae:.3f}")
    print(f"R²   : {r2:.3f}")
    return {'model': model_name, 'rmse': rmse, 'mae': mae, 'r2': r2}


In [45]:
# Exemple : utilisateur de 30 ans → colonne = 'Moyenne_Adulte 26-35'
age_group_col = get_age_group(30)

# Split des données
X_train, X_test, y_train, y_test = split_data_by_age_group(age_group_col)

# Modèle KNN
model_knn = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=5))
model_knn.fit(X_train, y_train)
results_knn = evaluate_model(model_knn, X_test, y_test, model_name='KNN')

# Modèle Ridge
model_ridge = make_pipeline(StandardScaler(), Ridge(alpha=1.0))
model_ridge.fit(X_train, y_train)
results_ridge = evaluate_model(model_ridge, X_test, y_test, model_name='Ridge Regression')


/Users/namiepajot/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


📊 Résultats KNN
RMSE : 1.575
MAE  : 1.233
R²   : -0.160
📊 Résultats Ridge Regression
RMSE : 2.127
MAE  : 1.594
R²   : -1.115


/Users/namiepajot/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [46]:
pd.DataFrame([results_knn, results_ridge])


,model,rmse,mae,r2
0,KNN,1.575236,1.232923,-0.159773
1,Ridge Regression,2.127346,1.594161,-1.115232


radom forest

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

def get_age_group(age):
    if age <= 12:
        return 'Moyenne_Enfant 1-12'
    elif age <= 17:
        return 'Moyenne_Adolescent 12-17'
    elif age <= 25:
        return 'Moyenne_Jeune adulte 18-25'
    elif age <= 35:
        return 'Moyenne_Adulte 26-35'
    elif age <= 50:
        return 'Moyenne_Adulte 36-50'
    else:
        return 'Moyenne_Senior 50+'

def compare_with_random_forest(user_age, test_size=0.2):
    age_group_col = get_age_group(user_age)
    mask = ~df[age_group_col].isna()
    X = genre_encoded[mask]
    y = df.loc[mask, age_group_col].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    results = []

    # KNN
    from sklearn.neighbors import KNeighborsRegressor
    model_knn = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=5))
    model_knn.fit(X_train, y_train)
    y_pred_knn = model_knn.predict(X_test)
    results.append({
        'Model': 'KNN',
        'RMSE': mean_squared_error(y_test, y_pred_knn, squared=False),
        'MAE': mean_absolute_error(y_test, y_pred_knn),
        'R2': r2_score(y_test, y_pred_knn)
    })

    # Ridge Regression
    from sklearn.linear_model import Ridge
    model_ridge = make_pipeline(StandardScaler(), Ridge(alpha=1.0))
    model_ridge.fit(X_train, y_train)
    y_pred_ridge = model_ridge.predict(X_test)
    results.append({
        'Model': 'Ridge Regression',
        'RMSE': mean_squared_error(y_test, y_pred_ridge, squared=False),
        'MAE': mean_absolute_error(y_test, y_pred_ridge),
        'R2': r2_score(y_test, y_pred_ridge)
    })

    # Random Forest
    model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    model_rf.fit(X_train, y_train)
    y_pred_rf = model_rf.predict(X_test)
    results.append({
        'Model': 'Random Forest',
        'RMSE': mean_squared_error(y_test, y_pred_rf, squared=False),
        'MAE': mean_absolute_error(y_test, y_pred_rf),
        'R2': r2_score(y_test, y_pred_rf)
    })

    return pd.DataFrame(results)

# Exemple d’appel
compare_with_random_forest(user_age=30)


/Users/namiepajot/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/namiepajot/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/namiepajot/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


,Model,RMSE,MAE,R2
0,KNN,1.575236,1.232923,-0.159773
1,Ridge Regression,2.127346,1.594161,-1.115232
2,Random Forest,1.543302,1.164710,-0.113227


KNN retravaillé

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def get_model_for_age_group(age_group_column):
    # On filtre les lignes qui ont une note dans cette tranche d’âge
    mask = ~df[age_group_column].isna()
    X = genre_encoded[mask]
    y = df.loc[mask, age_group_column].values

    # Définir les paramètres à tester
    param_grid = {
        'kneighborsregressor__n_neighbors': [3, 5, 7, 10],  # Essayer différentes valeurs de n_neighbors
        'kneighborsregressor__weights': ['uniform', 'distance'],  # Essayer avec et sans pondération des voisins
        'kneighborsregressor__metric': ['euclidean', 'manhattan']  # Tester différentes distances
    }

    # Utilisation de GridSearchCV pour optimiser les hyperparamètres
    model = make_pipeline(StandardScaler(), KNeighborsRegressor())
    grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)

    # Retourner le modèle optimal
    return grid_search.best_estimator_

def recommander_livres_knn_supervise(user_age, user_genres_principaux, user_genres_secondaires=[], top_k=5):
    # Déterminer la colonne cible correspondant à l'âge de l'utilisateur
    age_group_col = get_age_group(user_age)

    # Vérifier qu'il y a suffisamment de données
    if df[age_group_col].dropna().shape[0] < 5:
        raise ValueError(f"Pas assez de données pour la tranche d'âge {age_group_col}.")

    # Entraîner le modèle pour cette tranche avec optimisation d'hyperparamètres
    model = get_model_for_age_group(age_group_col)

    # Préparer les genres utilisateur
    user_genres = [g.lower() for g in user_genres_principaux + user_genres_secondaires]
    user_vector = mlb.transform([user_genres])
    user_input = user_vector.reshape(1, -1)

    # Prédire la note pour tous les livres de la tranche d’âge
    mask_valid = ~df[age_group_col].isna()
    X_all = genre_encoded[mask_valid]
    df_valid = df[mask_valid].copy()
    df_valid['predicted_rating'] = model.predict(X_all)

    # Filtrer les livres qui ont au moins un genre en commun
    df_valid = df_valid[df_valid['genre_combined'].apply(lambda g: any(gen in g for gen in user_genres))]

    # Trier par prédiction
    top_books = df_valid.sort_values(by='predicted_rating', ascending=False).head(top_k)

    return top_books[['Book-Title', 'Book-Author', 'Publisher', age_group_col, 'predicted_rating']]



In [49]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Fonction pour évaluer la performance du modèle
def evaluate_model(model, X_test, y_test):
    # Prédictions
    y_pred = model.predict(X_test)
    
    # Calcul des métriques
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return rmse, mae, r2

# Exemple d'évaluation pour une tranche d'âge (par exemple, "Moyenne_Adulte 26-35")
age_group_col = 'Moyenne_Adulte 26-35'  # Remplacer par la tranche d'âge de l'utilisateur

# Sélectionner les données pour la tranche d'âge
mask = ~df[age_group_col].isna()
X = genre_encoded[mask]
y = df.loc[mask, age_group_col].values

# Entraîner le modèle pour cette tranche avec GridSearchCV
model = get_model_for_age_group(age_group_col)

# Évaluer le modèle
rmse, mae, r2 = evaluate_model(model, X, y)

# Afficher les résultats
print(f"Performance du modèle KNN pour {age_group_col}:")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")


Performance du modèle KNN pour Moyenne_Adulte 26-35:
RMSE: 1.3873278103308928
MAE: 1.0630517035409577
R²: 0.13308745657976473


In [50]:
df.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Genre Principal', 'Genres Secondaires', 'Image-URL-L',
       'Moyenne_Adolescent 12-17', 'Moyenne_Adulte 26-35',
       'Moyenne_Adulte 36-50', 'Moyenne_Enfant 1-12',
       'Moyenne_Jeune adulte 18-25', 'Moyenne_Senior 50+', 'genre_combined'],
      dtype='object')